# Final Project of Quantum Computation

20195048 김현성

## Problem 1

Solve the following linear system with HHL algorithm

$$\begin{bmatrix}1 & -1/3 \\ -1/3 & 1\end{bmatrix} \mathbf{x} = \begin{bmatrix} 0 \\ 1 \end{bmatrix}$$

## Problem 2 

Extend the problem 1 result and solve following equation

$$\frac{1}{4}
\begin{bmatrix}
15 & 9 & 5 & -3 \\
9 & 15 & 3 & -5 \\
5 & 3 & 15 & -9 \\
-3 & -5 & -9 & 15\end{bmatrix} \mathbf{x} = 
\frac{1}{2} 
\begin{bmatrix}
1\\
1\\
1\\
1
\end{bmatrix}$$

## Answer

Since the matrix already a Hermit matrix, the calculation does not require a conversion. Before we implement HHL. Let us see the exact solutions of the each system.

### Exact solutions

In [14]:
from IPython.display import display, Math

In [15]:
import sympy as sp
from sympy.printing.mathml import mathml
sp.init_printing(use_unicode=True) # allow LaTeX printing


In [24]:
scale1 = 3
A1 = sp.Matrix([[3, -1], [-1, 3]])/scale1
b1 = sp.Matrix([[0], [1]])
scale2 = 4
A2 = sp.Matrix(
    [
        [15, 9, 5, -3], 
        [9, 15, 3, -5],
        [5, 3, 15, -9],
        [-3, -5, -9, 15]])/scale2
b2 = sp.Matrix(4*[[2]])/scale2

In [31]:
print("det(A_1): ", A1.det(), "det(A_2): ", A2.det())

det(A_1):  8/9 det(A_2):  64


In [32]:
A1.solve(b1).T

In [33]:
A2.solve(b2).T

### Implementation of HHL algorithm

In [ ]:
def initiate_matrix(A):
    #return A_gates
    pass
def HHL(A, wires):
    #return qc
    pass

### Results